In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [3]:
binance = Market.open("BN", "BTCBUSD")
Market.download(10)
binance

In [4]:
ohlcv = binance.ohlcv(DAYS_BEFORE(10), 0, 60*60)
ohlcv

,open,high,low,close,volume,count
timestamp,,,,,,
2022-12-04 04:00:00+00:00,16964.99,17022.27,16960.05,17008.82,2525.35745,44065.0
2022-12-04 05:00:00+00:00,17008.79,17071.90,17000.00,17016.02,5337.57107,90863.0
2022-12-04 06:00:00+00:00,17016.05,17046.92,16995.48,16999.70,2971.78116,58160.0
2022-12-04 07:00:00+00:00,17000.03,17017.10,16991.26,16997.22,2907.06426,57056.0
2022-12-04 08:00:00+00:00,16997.24,17050.17,16976.96,17014.84,4886.35794,84342.0
...,...,...,...,...,...,...
2022-12-12 19:00:00+00:00,17026.59,17048.62,17007.44,17045.25,3574.80175,72390.0
2022-12-12 20:00:00+00:00,17045.48,17127.99,17045.27,17125.78,6467.04636,121706.0
2022-12-12 21:00:00+00:00,17125.49,17220.79,17122.76,17176.70,8703.74854,158110.0


In [5]:
chart = Chart(900, 400, ohlcv)
chart.show()

Loading BokehJS ...

In [6]:


class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            super().__init__()
            self.K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*10
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        if session.short_order_size or session.long_order_size: #全開の注文処理中の場合はなにもしない       
            return 
        
        ohlcv_df = session.ohlcv(60*60*2, 6)       # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean() * self.K     #　過去５本足のレンジの平均値 * K

        # Long/Short判定
        diff_low   =   (ohlcv_latest["open"][0] - ohlcv_latest["low"][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest["open"][0] - ohlcv_latest["high"][0])  
        detect_long = range_width  < diff_high
        
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range', time_us, range_width)

        #　執行方法（順方向のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            if not session.long_position_size:
                if session.short_position_size:
                    session.place_order("Buy", session.best_buy_price, 0.02, 60*30, "doten Long")                        
                else:
                    session.place_order("Buy", session.best_buy_price, 0.01, 60*30, "Open Long")    
            else:
                pass

        if detect_short:
            if not session.short_position_size:
                if session.long_position_size:
                    session.place_order("Sell", session.best_sell_price, 0.02, 60*30, "Doten Short")                     
                else:
                    session.place_order("Sell", session.best_sell_price, 0.01, 60*30, "Open Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [7]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%

In [8]:
agent = Agent()
back_tester.run(agent, rbot.DAYS_BEFORE(10), 0)


,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-05 00:50:00+00:00,2022-12-05 00:50:56.656000+00:00,0000-0001,0,Buy,True,Open,17219.16,0.01,172.1916,0.00,...,17219.16,0.01,172.1916,0.0000,0.172192,-0.172192,0.01,Open Long,-0.172192,0.01
2022-12-05 15:10:00+00:00,2022-12-05 15:40:00.146000+00:00,0000-0002,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17207.99,0.02,344.1598,0.0000,0.000000,0.000000,0.00,Doten Short,-0.172192,0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0003,0,Sell,True,Close,17219.16,0.01,172.1916,17088.30,...,17088.30,0.01,170.8830,-1.3086,0.170883,-1.479483,-0.01,Doten Short,-1.651675,0.00
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0003,1,Sell,True,Open,17088.30,0.01,170.8830,0.00,...,17088.30,0.01,170.8830,0.0000,0.170883,-0.170883,-0.01,Doten Short,-1.822558,-0.01
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0004,0,Buy,True,Close,17088.30,0.01,170.8830,17067.54,...,17067.54,0.01,170.6754,0.2076,0.170675,0.036925,0.01,doten Long,-1.785633,0.00
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0004,1,Buy,True,Open,17067.54,0.01,170.6754,0.00,...,17067.54,0.01,170.6754,0.0000,0.170675,-0.170675,0.01,doten Long,-1.956308,0.01
2022-12-07 07:20:00+00:00,2022-12-07 07:50:00.306000+00:00,0000-0005,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,16934.06,0.02,338.6812,0.0000,0.000000,0.000000,0.00,Doten Short,-1.956308,0.01
2022-12-09 13:40:00+00:00,2022-12-09 14:10:00.314000+00:00,0000-0006,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17233.61,0.02,344.6722,0.0000,0.000000,0.000000,0.00,Doten Short,-1.956308,0.01
2022-12-11 20:30:00+00:00,2022-12-11 20:47:03.013000+00:00,0000-0007,0,Sell,True,Close,17067.54,0.01,170.6754,17136.00,...,17136.00,0.01,171.3600,0.6846,0.171360,0.513240,-0.01,Doten Short,-1.443068,0.00


In [9]:
back_tester

Exchange name,BN
Market name,BTCBUSD
Size in price currency,False
maker fee rate,0.1 [%]
Agent class name,Agent
enable: on_tick,False
enable: on_clock,True
enable: on_update,True
clock interval:,600 [sec]
start,"2022-12-04T04:24:57.359000 (1,670,127,897,359,000)"
end,"2022-12-12T23:59:59.684000 (1,670,889,599,684,000)"


In [11]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(back_tester.result)

chart.show()

Loading BokehJS ...

In [ ]:
agent.get_indicator('diff_low')

,value
timestamp,
2022-12-04 04:10:00+00:00,0.09
2022-12-04 04:20:00+00:00,0.09
2022-12-04 04:30:00+00:00,0.52
2022-12-04 04:40:00+00:00,0.52
2022-12-04 04:50:00+00:00,0.52
...,...
2022-12-12 23:20:00+00:00,33.75
2022-12-12 23:30:00+00:00,33.75
2022-12-12 23:40:00+00:00,33.75


In [ ]:
chart = Chart(900, 400, ohlcv)

chart.draw_result(r)

chart.new_figure("indicator_panel", 150, 'indicator')
chart.line('indicator_panel', agent.indicator('diff_low'), 'timestamp', 'value', 'diff_low', '#00ff00')
chart.line('indicator_panel', agent.indicator('diff_high'), 'timestamp', 'value', 'diff_high', '#ffff00')
chart.line('indicator_panel', agent.indicator('range'), 'timestamp', 'value', 'range', '#00ffff')

chart.show()

Loading BokehJS ...

price
volume
profit
position
indicator_panel
